In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib
from time import sleep
import requests
import concurrent
from tqdm import tqdm

In [2]:
def urls():
    url_centro = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+regiao-do-centro/'
    url_sul    = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-sul/'
    url_norte  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-norte/'
    url_oeste  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-oeste/'
    url_leste  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-leste/'

    return [url_centro,url_sul,url_norte ,url_oeste,url_leste ]

In [3]:
def criaHeader():
    headers = requests.utils.default_headers()
    headers.update({
        "User-Agent":"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"
    })
    return headers

In [4]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in np.arange(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [5]:
# vamos pegar o numero de paginas da url
def numPag(url, header):
    lista_retorno = []
    for url_site in  tqdm(url):
        r = requests.get(str(url_site), headers=header)
        soup = BeautifulSoup(r.content, 'lxml')
        lista_retorno.append(int(soup.find('span',{'class':'pull-right num-of'}).text.replace('de ', '').replace('.', '')))
        sleep(2)
    # numero de paginas
    return lista_retorno

In [6]:
def soup_url(url, num_pag):
    lista = []
    if type(url) is list:
        ij=0
        for num in tqdm(num_pag):
            for i in range(1, num+1):
                url_compl = '#{"pagina":"'+str(i)+'","formato":"Lista"}'
                lista.append(url[ij] + url_compl)
            ij=ij+1
    else:
        for i in tqdm(range(1, num_pag+1)):
            url_compl = '#{"pagina":"'+str(i)+'","formato":"Lista"}'
            lista.append(url + url_compl)
    return np.array(lista)

In [7]:
def retornaSoup(url, header):
    r = requests.get(url, headers=header)
    sleep(3)
    return r.content

In [8]:
def retornaBSoup(html):
    return BeautifulSoup(html, 'lxml')

In [9]:
def salvaListaComprimida(lista):
    np.savez_compressed('save', lista)

In [10]:
def carregaListaComprimida(arquivo):
    with np.load(arquivo) as data:
        a = data.items()
    return a[0][1]

In [11]:
# retorna uma lista com o html de todas as paginas
def retornaListaHTML(lista_url):
    lista_html = []
    tamanho_batch = round((len(lista_url)/100) * 8)
    total = round(len(lista_url)/tamanho_batch)

    for lista in tqdm(batch(lista_url, tamanho_batch),total=total):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            result = executor.map(retornaSoup, lista)
            lista_html.append(list(result))
    return lista_html

In [12]:
lista_html = []
lista_html = carregaListaComprimida('save.npz')

In [13]:
url_lista = urls()

In [17]:
header = criaHeader()

In [18]:
lista_numPag = numPag(url_lista, header)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.46s/it]


In [19]:
url_lista_todos = soup_url(url_lista, lista_numPag)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 713.51it/s]


In [20]:
if len(lista_html) == 0:
    lista_html_aux = retornaListaHTML(url_lista_todos)
    lista_html = [i for j in lista_html for i in j]

In [ ]:
lista_soup = []
for lista in tqdm(batch(lista_html, 200),total=(len(lista_html)/200)):
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        lista_soup.append(executor.map(retornaBSoup, lista))


  0%|                                                                                        | 0/16.32 [00:00<?, ?it/s]
  6%|████▉                                                                           | 1/16.32 [00:54<13:54, 54.48s/it]
 12%|█████████▊                                                                      | 2/16.32 [01:56<13:52, 58.14s/it]
 18%|██████████████▋                                                                 | 3/16.32 [02:50<12:35, 56.72s/it]
 25%|███████████████████▌                                                            | 4/16.32 [03:55<12:06, 58.97s/it]
 31%|████████████████████████▌                                                       | 5/16.32 [04:57<11:12, 59.41s/it]
 37%|█████████████████████████████▍                                                  | 6/16.32 [05:49<10:01, 58.33s/it]
 43%|██████████████████████████████████▎                                             | 7/16.32 [06:59<09:18, 59.97s/it]
 49%|██████████████████████████████████